In [2]:
import os, sys
import subprocess
from utilities_pipeline_status import *
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager_v2 import DataManager
from a_driver_utilities import *

from a_GUI_utilities_pipeline_status import *

print all_stacks

Setting environment for Precision WorkStation for Alex Newberry


No vtk

[521 1057 128 465 None None]
[521 1057 128 465 None None]
[564 1088 102 736 None None]
[564 1088 102 736 None None]
[521 1057 128 465 None None]
[521 1057 128 465 None None]
['MD6635TIFF', 'DK17CZI', 'DK23', 'DK19']


In [106]:
stack = 'DK23'

pipeline_status = get_pipeline_status( stack )
print(pipeline_status)

a_script_preprocess_4


In [114]:
def apply_transform( stack, T, input_fp, output_fp=None, as_thumb_crop=False ):
    """
    Applies transform T onto the image at img_fp and return the result
    """
    
    #version = get_version(stack)
    
    #img_fp = DataManager.get_image_filepath_v2(stack=stack, section=metadata_cache['filenames_to_sections'][stack][fn], 
                            #prep_id='None', resol='thumbnail', version=version)
    img_fp = input_fp
    
    op_str = ''
    op_str += " +distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' " % {
     'sx':T[0,0],
     'sy':T[1,1],
     'rx':T[1,0],
     'ry':T[0,1],
     'tx':T[0,2],
     'ty':T[1,2],}
    
    if as_thumb_crop:
        x = 0
        y = 0
        w = 2001
        h = 1001
        op_str += ' -crop %(w)sx%(h)s%(x)s%(y)s\! ' % {
                         'x': '+' + str(x) if int(x) >= 0 else str(x),
                         'y': '+' + str(y) if int(y) >= 0 else str(y),
                         'w': str(w),
                         'h': str(h)}
    
    if output_fp==None:
        output_fp_root = os.path.join( os.environ['ROOT_DIR'], 'CSHL_data_processed', stack, 'tmp')
        if not os.path.exists( output_fp_root ):
            os.makedirs(output_fp_root)
        output_fp = os.path.join( output_fp_root, 'tmp_'+fn+'.tif' )
        delete_after = True
    else:
        if not os.path.exists( os.path.dirname(output_fp) ):
            os.makedirs( os.path.dirname(output_fp) )
        delete_after = False

    try:
        bg_color = 'black' #get_padding_color(stack)
        
        execute_command("convert \"%(input_fp)s\"  +repage -virtual-pixel background \
-background %(bg_color)s %(op_str)s -flatten -compress lzw \"%(output_fp)s\"" % \
                    {'op_str': op_str,
                     'input_fp': img_fp,
                     'output_fp': output_fp,
                     'bg_color': bg_color})
        
        img = cv2.imread( output_fp )
        if delete_after:
            os.remove( output_fp ) 
            pass
        return img
    except Exception as e:
        print e
        return None
    
def get_anchor_transform( stack, fn, downsampled=True ):
    if downsampled:
        T = DataManager.load_transforms(stack, downsample_factor=32, use_inverse=True)[fn]
    else:
        T = DataManager.load_transforms(stack, use_inverse=True)[fn]
    return T

In [115]:
# Transform thumbnail files!
T_list = DataManager.load_transforms_v2(stack, in_image_resolution='thumbnail', out_image_resolution='thumbnail', use_inverse=False)
for fn in metadata_cache['filenames_to_sections'][stack].keys():
    fn_secondary_C = fn.replace('C0','C1').replace('CFP','TuYFP') + '_raw.tif'
    
    fp_secondary_C = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_C0', fn_secondary_C)
    fp_secondary_C_thumbnail = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_thumbnail_C0', fn_secondary_C)
    fp_secondary_C_output = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_prep1_thumbnail_C0', fn_secondary_C)
    
    # Load anchor transform
    T = T_list[fn]
    
    apply_transform( stack, T, input_fp=fp_secondary_C_thumbnail, output_fp=fp_secondary_C_output, as_thumb_crop=True )

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide011_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998219,-0.059649,0.059649,0.998219,301.218910,113.662715'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide011_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide029_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998287,-0.058511,0.058511,0.998287,200.282869,81.604435'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide029_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide059_2019_09_11_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999910,-0.013440,0.013440,0.999910,-42.353697,-3.692041'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide059_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide076_2019_09_03_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.996992,-0.077499,0.077499,0.996992,-50.179097,78.917440'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide076_2019_09_03_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Roo

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide114_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.964831,-0.262871,0.262871,0.964831,271.497959,255.796258'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide114_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide108_2019_09_10_S07_S6_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999502,-0.031568,0.031568,0.999502,303.499592,49.499658'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide108_2019_09_10_S07_S6_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Te

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide120_2019_09_10_S11_S10_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.960583,-0.277994,0.277994,0.960583,404.190164,323.985912'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide120_2019_09_10_S11_S10_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide078_2019_09_11_S15_S14_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998689,-0.051193,0.051193,0.998689,-60.061119,76.962121'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide078_2019_09_11_S15_S14_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atla

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide039_2019_09_10_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998577,-0.053336,0.053336,0.998577,142.546556,27.713972'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide039_2019_09_10_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide071_2019_09_11_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.994578,-0.103995,0.103995,0.994578,111.535363,87.781853'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide071_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Roo

...

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide067_2019_09_11_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999906,-0.013690,0.013690,0.999906,-12.906680,20.148150'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide067_2019_09_11_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide014_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.993886,-0.110410,0.110410,0.993886,306.794328,134.970430'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide014_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide078_2019_09_11_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.992033,-0.125980,0.125980,0.992033,-86.082029,104.076829'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide078_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide023_2019_09_10_S07_S6_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999903,-0.013952,0.013952,0.999903,251.320154,69.810725'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide023_2019_09_10_S07_S6_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Ro

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide105_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998526,-0.054274,0.054274,0.998526,234.308762,103.278454'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide105_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide030_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.994999,-0.099887,0.099887,0.994999,160.660832,100.585932'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide030_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide023_2019_09_10_S19_S18_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999830,0.018445,-0.018445,0.999830,250.886632,45.701426'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide023_2019_09_10_S19_S18_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide116_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.980603,-0.196003,0.196003,0.980603,294.413394,216.055992'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide116_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atla

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide051_2019_09_11_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999236,0.039088,-0.039088,0.999236,27.178357,-27.505773'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide051_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_thumbnail_C0/DK23_slide007_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.997785,-0.066518,0.066518,0.997785,408.712234,213.473900'  -crop 2001x1001+0+0\!  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide007_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_

In [47]:
# Transform raw files!
T_list = DataManager.load_transforms_v2(stack, in_image_resolution='raw', out_image_resolution='raw', use_inverse=True)

for fn in metadata_cache['filenames_to_sections'][stack].keys():#[205:206]:
    fn_secondary_C = fn.replace('C0','C1').replace('CFP','TuYFP') + '_raw.tif'
    
    fp_secondary_C = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_C0', fn_secondary_C)
    fp_secondary_C_output = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_prep1_raw_C0', fn_secondary_C)
    
    # Load anchor transform
    T = T_list[fn]
    
    apply_transform( stack, T, input_fp=fp_secondary_C, output_fp=fp_secondary_C_output )

convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide011_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998219,0.059649,-0.059649,0.998219,-9404.888083,-4205.683502'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide011_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide029_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998287,0.058511,-0.058511,0.998287,-6245.280652,-2981.865361'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide029_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide081_2019_09_11

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide076_2019_09_03_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.996992,0.077499,-0.077499,0.996992,1796.614976,-2393.319965'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide076_2019_09_03_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide015_2019_09_10_S19_S18_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999947,0.010293,-0.010293,0.999947,-9928.245469,-3034.061603'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide015_2019_09_10_S19_S18_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide051_2019_09_11_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999895,0.014479,-0.014479,0.999895,536.481168,-716.085342'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide051_2019_09_11_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide028_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999944,-0.010599,0.010599,0.999944,-7129.230183,-1779.336888'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide028_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide102_

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide120_2019_09_10_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.971129,0.238553,-0.238553,0.971129,-8445.448281,-13440.070951'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide120_2019_09_10_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide049_2019_09_11_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.995398,0.095825,-0.095825,0.995398,974.330851,-2112.186982'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide049_2019_09_11_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide03

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide015_2019_09_10_S07_S6_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998796,0.049053,-0.049053,0.998796,-8819.414726,-4125.782565'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide015_2019_09_10_S07_S6_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide060_2019_09_11_S01_S0_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '1.000000,0.000190,-0.000190,1.000000,-44.880883,45.381830'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide060_2019_09_11_S01_S0_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide039_20

...

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide067_2019_09_11_S08_S7_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999906,0.013690,-0.013690,0.999906,421.801271,-639.026414'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide067_2019_09_11_S08_S7_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide014_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.993886,0.110410,-0.110410,0.993886,-9280.526947,-5376.592851'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide014_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide02

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide023_2019_09_10_S07_S6_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999903,0.013952,-0.013952,0.999903,-8010.295221,-2345.927544'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide023_2019_09_10_S07_S6_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide011_2019_09_10_S07_S6_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.990676,0.136242,-0.136242,0.990676,-8931.913518,-6756.097331'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide011_2019_09_10_S07_S6_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide0

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide065_2019_09_11_S15_S14_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999981,0.006173,-0.006173,0.999981,1585.050628,-271.421775'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide065_2019_09_11_S15_S14_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide061_2019_09_11_S15_S14_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999674,-0.025543,0.025543,0.999674,-4361.762877,1079.558140'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide061_2019_09_11_S15_S14_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide098_2019_09_10_S21_S20_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999708,-0.024157,0.024157,0.999708,-5813.748457,-226.715735'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide098_2019_09_10_S21_S20_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide028_2019_09_10_S13_S12_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.999682,0.025220,-0.025220,0.999682,-6874.676087,-2427.586681'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide028_2019_09_10_S13_S12_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_sli

return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide044_2019_09_11_S15_S14_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998913,0.046604,-0.046604,0.998913,-3776.091587,-1473.429713'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide044_2019_09_11_S15_S14_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_slide001_2019_09_10_S09_S8_C1_TuYFP_raw.tif"  +repage -virtual-pixel background -background black  +distort AffineProjection '0.998685,-0.051266,0.051266,0.998685,-17478.013860,-8015.827157'  -flatten -compress lzw "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_raw_C0/DK23_slide001_2019_09_10_S09_S8_C1_TuYFP_raw.tif"
return code: 0
convert "/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_C0/DK23_sli

In [95]:
! mkdir /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel/

# Transform raw files!
T_list = DataManager.load_transforms_v2(stack, in_image_resolution='raw', out_image_resolution='raw', use_inverse=True)

for fn in metadata_cache['filenames_to_sections'][stack].keys():
    fn_secondary_C = fn.replace('C0','C1').replace('CFP','TuYFP') + '_raw.tif'
    
    fp_secondary_C = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_raw_C0', fn_secondary_C)
    fp_secondary_C_output = os.path.join( DataManager.get_images_root_folder( stack ), stack+'_prep1_raw_C0', fn_secondary_C)
    
    # Load anchor transform
    T = T_list[fn]
    
    apply_transform( stack, T, input_fp=fp_secondary_C, output_fp=fp_secondary_C_output )

! mkdir /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel/

In [98]:
! python a_script_preprocess_4.py DK23 NTB 

Setting environment for Precision WorkStation for Alex Newberry
No mcubes
[521 1057 128 465 None None]
[521 1057 128 465 None None]
[564 1088 102 736 None None]
[564 1088 102 736 None None]
[521 1057 128 465 None None]
[521 1057 128 465 None None]
Setting environment for Precision WorkStation for Alex Newberry
No mcubes
[521 1057 128 465 None None]
[521 1057 128 465 None None]
[564 1088 102 736 None None]
[564 1088 102 736 None None]
[521 1057 128 465 None None]
[521 1057 128 465 None None]
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide011_2019_09_10_S13_S12_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide029_2019_09_10_S01_S0_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide081_2019_09_11_S15_S14_C0_CFP_thumbnail_NtbN

Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide012_2019_09_10_S07_S6_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide022_2019_09_10_S07_S6_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide004_2019_09_10_S06_S5_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide006_2019_09_10_S06_S5_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide112_2019_09_11_S07_S6_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide002_2019_09_10_S06_S5_C0_

Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide113_2019_09_10_S19_S18_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide074_2019_09_11_S15_S14_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide026_2019_09_10_S08_S7_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide053_2019_09_11_S08_S7_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide089_2019_09_10_S08_S7_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide002_2019_09_10_S16_S15_

Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide017_2019_09_11_S19_S18_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide009_2019_09_10_S07_S6_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide109_2019_09_10_S13_S12_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide100_2019_09_10_S08_S7_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide064_2019_09_11_S01_S0_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide019_2019_09_11_S13_S12_

Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide026_2019_09_10_S01_S0_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide028_2019_09_10_S07_S6_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide111_2019_09_10_S07_S6_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide112_2019_09_11_S19_S18_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide010_2019_09_10_S13_S12_C0_CFP_thumbnail_NtbNormalized.tif
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_NtbNormalized/DK23_slide007_2019_09_10_S07_S6_C

...

Rescale mask: 8.30 seconds.
Compute mean/std for sample regions: 3.53 seconds.
Interpolate mean map: 2.23 seconds.
Scale up mean map: 5.31 seconds.
Interpolate std map: 2.16 seconds.
Scale up std map: 5.13 seconds.
Normalize: 3.11 seconds.
Save float version: 2.35 seconds.
Save float histogram: 5.05 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_Ntb/DK23_slide113_2019_09_10_S13_S12_C0_CFP_raw_Ntb.tif
Load image: 3.72 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_mask/DK23_slide113_2019_09_10_S13_S12_C0_CFP_thumbnail_mask.png
Rescale mask: 11.41 seconds.
Compute mean/std for sample regions: 5.64 seconds.
Interpolate mean map: 2.99 seconds.
Scale up mean map: 7.13 seconds.
Interpolate std map: 2.96 seconds.
Scale up std map: 7.06 seconds.
Normalize: 4.76 seconds.
Save float version: 3.12 seconds.
Save float histogram: 7.37 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_d

Rescale mask: 19.18 seconds.
Compute mean/std for sample regions: 10.52 seconds.
Interpolate mean map: 4.55 seconds.
Scale up mean map: 12.27 seconds.
Interpolate std map: 4.56 seconds.
Scale up std map: 12.28 seconds.
Normalize: 9.91 seconds.
Save float version: 5.32 seconds.
Save float histogram: 13.30 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_Ntb/DK23_slide058_2019_09_11_S15_S14_C0_CFP_raw_Ntb.tif
Load image: 12.20 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_mask/DK23_slide058_2019_09_11_S15_S14_C0_CFP_thumbnail_mask.png
Rescale mask: 43.51 seconds.
Compute mean/std for sample regions: 21.68 seconds.
Interpolate mean map: 10.81 seconds.
Scale up mean map: 29.18 seconds.
Interpolate std map: 11.09 seconds.
Scale up std map: 32.70 seconds.
Normalize: 22.30 seconds.
Save float version: 11.78 seconds.
Save float histogram: 28.24 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_

Rescale mask: 37.15 seconds.
Compute mean/std for sample regions: 20.09 seconds.
Interpolate mean map: 9.90 seconds.
Scale up mean map: 24.91 seconds.
Interpolate std map: 10.85 seconds.
Scale up std map: 27.39 seconds.
Normalize: 24.17 seconds.
Save float version: 12.25 seconds.
Save float histogram: 26.67 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_Ntb/DK23_slide011_2019_09_10_S01_S0_C0_CFP_raw_Ntb.tif
Load image: 4.24 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_mask/DK23_slide011_2019_09_10_S01_S0_C0_CFP_thumbnail_mask.png
Rescale mask: 13.21 seconds.
Compute mean/std for sample regions: 6.85 seconds.
Interpolate mean map: 3.44 seconds.
Scale up mean map: 8.20 seconds.
Interpolate std map: 3.38 seconds.
Scale up std map: 8.19 seconds.
Normalize: 7.26 seconds.
Save float version: 3.77 seconds.
Save float histogram: 8.65 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/

Rescale mask: 26.33 seconds.
Compute mean/std for sample regions: 15.22 seconds.
Interpolate mean map: 7.02 seconds.
Scale up mean map: 16.21 seconds.
Interpolate std map: 6.98 seconds.
Scale up std map: 16.15 seconds.
Normalize: 15.04 seconds.
Save float version: 8.88 seconds.
Save float histogram: 19.41 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_Ntb/DK23_slide023_2019_09_10_S13_S12_C0_CFP_raw_Ntb.tif
Load image: 14.29 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_mask/DK23_slide023_2019_09_10_S13_S12_C0_CFP_thumbnail_mask.png
Rescale mask: 24.74 seconds.
Compute mean/std for sample regions: 11.18 seconds.
Interpolate mean map: 5.85 seconds.
Scale up mean map: 13.59 seconds.
Interpolate std map: 5.75 seconds.
Scale up std map: 13.55 seconds.
Normalize: 12.03 seconds.
Save float version: 6.08 seconds.
Save float histogram: 13.83 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Te

Rescale mask: 25.24 seconds.
Compute mean/std for sample regions: 11.85 seconds.
Interpolate mean map: 6.95 seconds.
Scale up mean map: 15.06 seconds.
Interpolate std map: 6.03 seconds.
Scale up std map: 14.47 seconds.
Normalize: 11.33 seconds.
Save float version: 6.28 seconds.
Save float histogram: 14.42 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_raw_Ntb/DK23_slide006_2019_09_10_S01_S0_C0_CFP_raw_Ntb.tif
Load image: 2.11 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_thumbnail_mask/DK23_slide006_2019_09_10_S01_S0_C0_CFP_thumbnail_mask.png
Rescale mask: 6.12 seconds.
Compute mean/std for sample regions: 3.10 seconds.
Interpolate mean map: 1.51 seconds.
Scale up mean map: 3.66 seconds.
Interpolate std map: 1.49 seconds.
Scale up std map: 3.68 seconds.
Normalize: 2.42 seconds.
Save float version: 1.64 seconds.
Save float histogram: 3.94 seconds.
Trying to load /data2/Atlas_Root_Dirs/Beta_Testing/CSH

In [96]:
metadata_cache['sections_to_filenames'][stack].keys()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [102]:
stack = 'DK23'
stain = "Ntb"

In [103]:
# This next line replaces the user step for  manually finding this cropbox
rostral_lim, caudal_lim, dorsal_lim, ventral_lim = get_prep5_limits_from_prep1_thumbnail_masks(stack)

command = ['python', 'a_script_preprocess_5.py', stack, stain, '-l', 
          str(rostral_lim), str(caudal_lim), str(dorsal_lim), str(ventral_lim)]
call_and_time( command, completion_message='')

('rostral:', 0)
('caudal:', 1528.0)
('dorsal:', 56.0)
('ventral:', 792.0)
**************************************************************************************************

Script a_script_preprocess_5.py completed. Took  587.5  seconds


**************************************************************************************************


In [53]:
metadata_cache['filenames_to_sections'][stack].keys().sort()

None


In [78]:
def closest_valid_fn(fn, fns):
    fn_index = fns.index(fn)
    
    for dist in range(1,20):
        # Check higher index
        try:
            fp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/'+fns[fn_index+dist]

            if os.path.exists(fp):
                if len(os.listdir(fp) ) != 0:
                    return fns[fn_index+dist]
        except:
            pass
        # Check lower index
        try:
            fp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/'+fns[fn_index-dist]
            
            if os.path.exists(fp):
                if len(os.listdir(fp) ) != 0:
                    return fns[fn_index-dist]
        except:
            pass
    
fns = metadata_cache['filenames_to_sections'][stack].keys()
fns.sort()
for fn in fns:
    fp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/'+fn
    if not os.path.exists(fp):
        os.makedirs(fp)
        
    if len(os.listdir(fp) ) == 0:
        
        # Copy another file from before or after
        try:
            fn_to_copy = closest_valid_fn(fn, fns)
            fp_to_copy_1 = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/'+fn_to_copy+'/'+\
            fn_to_copy+'_prep1_thumbnail_autoSubmask_0.png'
            fp_to_copy_2 = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/'+fn_to_copy+'/'+\
            fn_to_copy+'_prep1_thumbnail_autoSubmaskDecisions.csv'
            
            fp_to_paste_1 = fp_to_copy_1.replace( fn_to_copy, fn)
            fp_to_paste_2 = fp_to_copy_2.replace( fn_to_copy, fn)
            
            print('cp '+fp_to_copy_1+' '+fp_to_paste_1)
            ! cp $fp_to_copy_1 $fp_to_paste_1
            ! cp $fp_to_copy_2 $fp_to_paste_2
        except Exception as e:
            print(str(e))
            pass
    else:
        continue

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/DK23_slide120_2019_09_10_S01_S0_C0_CFP/DK23_slide120_2019_09_10_S01_S0_C0_CFP_prep1_thumbnail_autoSubmask_0.png /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/DK23_slide120_2019_09_10_S06_S5_C0_CFP/DK23_slide120_2019_09_10_S06_S5_C0_CFP_prep1_thumbnail_autoSubmask_0.png
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/DK23_slide120_2019_09_10_S06_S5_C0_CFP/DK23_slide120_2019_09_10_S06_S5_C0_CFP_prep1_thumbnail_autoSubmask_0.png /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_autoSubmasks/DK23_slide120_2019_09_10_S11_S10_C0_CFP/DK23_slide120_2019_09_10_S11_S10_C0_CFP_prep1_thumbnail_autoSubmask_0.png


In [124]:
# Copy SECONDARY Channel
fns = metadata_cache['filenames_to_sections'][stack].keys()
fns.sort()

rfp = os.path.join( os.environ['ROOT_DIR'], 'CSHL_data_processed', stack)

for fn in fns:
    fn_C0 = fn.replace( 'C0_CFP', 'C1_TuYFP' )
    section = metadata_cache['filenames_to_sections'][stack][fn]
    section_str = str(section).zfill(3)
    
    
    input_fn = os.path.join( rfp, stack+'_prep1_thumbnail_C0', fn_C0+'_raw.tif' )
    output_fn = os.path.join( rfp, stack+'_Neuroglancer_secondary_channel_thumbnail_input', section_str+'.tif' )
    
    ! cp $input_fn $output_fn
    #print(input_fn)
    #print(output_fn)

In [125]:
# Copy PRIMARY Channel
fns = metadata_cache['filenames_to_sections'][stack].keys()
fns.sort()

rfp = os.path.join( os.environ['ROOT_DIR'], 'CSHL_data_processed', stack)

for fn in fns:
    section = metadata_cache['filenames_to_sections'][stack][fn]
    section_str = str(section).zfill(3)
    
    
    input_fn = os.path.join( rfp, stack+'_prep1_thumbnail_NtbNormalized', fn+'_prep1_thumbnail_NtbNormalized.tif' )
    output_fn = os.path.join( rfp, stack+'_Neuroglancer_primary_channel_thumbnail_input', section_str+'.tif' )
    
    ! cp $input_fn $output_fn

In [119]:
os.path.join( os.environ['ROOT_DIR'], 'CSHL_data_processed', stack)

'/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23'

In [118]:
fn

'DK23_slide120_2019_09_10_S11_S10_C0_CFP'